In [100]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [101]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [102]:
tf.keras.backend.clear_session()

In [103]:
subject = 'Roraima - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [104]:
data = pd.read_csv('2003_mo_model_input_RR.csv')
# data = add_cos_monthly(data, "Sazonalidade")
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - Desemprego,Roraima - value,Roraima - Consumo de Cimento (t)
0,2003-1,0.734916,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,6.128460e+06,332580.202821,12.101540,5.339136e+06,8.263084,0.294583,6.920
1,2003-2,0.734957,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,6.136622e+06,332914.755543,12.103485,5.342057e+06,8.257123,0.296505,7.476
2,2003-3,0.734998,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,6.144784e+06,333249.308265,12.105429,5.344978e+06,8.251161,0.298178,8.659
3,2003-4,0.735038,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,6.152947e+06,333583.860987,12.107373,5.347899e+06,8.245199,0.299598,7.259
4,2003-5,0.735079,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,6.161109e+06,333918.413709,12.109318,5.350821e+06,8.239238,0.300965,6.786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.581560,9.133
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.579849,10.622
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.577718,10.300
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.575950,10.258


In [105]:
input_data = data.iloc[:split_index + 1,1:-1]
# mean = np.mean(input_data, axis=0)
# stddev =  np.std(input_data, axis=0)
# input_data = ((input_data - mean) /stddev)
scaler=MinMaxScaler()
scaled_data = scaler.fit_transform(input_data)
input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - Desemprego,Roraima - value
0,0.747334,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.144883,0.000000,0.096219,0.233105
1,0.753811,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,0.005680,0.019675,0.157302,0.007541,0.094065,0.242820
2,0.760287,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,0.011361,0.039350,0.169722,0.015083,0.091911,0.251273
3,0.766763,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,0.017041,0.059025,0.182142,0.022624,0.089756,0.258452
4,0.773239,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,0.022721,0.078700,0.194561,0.030165,0.087602,0.265360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.132632,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.981357,0.451779,0.025853,0.979634,0.911276,0.956241
188,0.099474,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.978694,0.450751,0.019390,0.976725,0.904894,0.965933
189,0.066316,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.976030,0.449723,0.012926,0.973815,0.898512,0.976605
190,0.033158,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.973367,0.448695,0.006463,0.970906,0.892130,0.988214


In [106]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      5.753
1      5.006
2      5.470
3      1.513
4      3.706
       ...  
235      NaN
236      NaN
237      NaN
238      NaN
239      NaN
Name: Roraima - Consumo de Cimento (t), Length: 240, dtype: float64

In [107]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - Desemprego,Roraima - value
0,0.747334,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.144883,0.000000,0.096219,0.233105
1,0.753811,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,0.005680,0.019675,0.157302,0.007541,0.094065,0.242820
2,0.760287,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,0.011361,0.039350,0.169722,0.015083,0.091911,0.251273
3,0.766763,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,0.017041,0.059025,0.182142,0.022624,0.089756,0.258452
4,0.773239,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,0.022721,0.078700,0.194561,0.030165,0.087602,0.265360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.767059,0.379453,0.024030,0.338208,0.039971,0.890665,0.956993,0.182507,0.966672,0.463988,0.260471,0.948715,0.993057,0.839909
158,0.746754,0.332832,0.021531,0.371581,0.027548,0.893949,0.953662,0.168812,0.969378,0.461505,0.258059,0.952704,0.986114,0.841507
159,0.726448,0.313954,0.019385,0.365349,0.037038,0.897232,0.950331,0.152959,0.972083,0.459022,0.255648,0.956693,0.979171,0.842961
160,0.706143,0.294856,0.017671,0.404993,0.070841,0.900516,0.947000,0.137613,0.974789,0.456539,0.253236,0.960683,0.972228,0.844170


In [108]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       5.753
1       5.006
2       5.470
3       1.513
4       3.706
        ...  
157     6.557
158     8.184
159     8.092
160     7.806
161    10.831
Name: Roraima - Consumo de Cimento (t), Length: 162, dtype: float64

In [109]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [110]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 14)

In [111]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Roraima - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Roraima - PIB - Estadual,Roraima - PIB - Construção Civil,Roraima - PIB - Per Capita,Roraima - PIB - Preços de Mercado,Roraima - Desemprego,Roraima - value
126,0.894896,0.612440,0.052718,0.094523,0.201912,0.780557,0.994213,0.372054,0.811538,0.844319,0.573064,0.823394,0.702090,0.653168
127,0.892606,0.626227,0.056653,0.200555,0.209545,0.784414,0.995177,0.375438,0.817887,0.818372,0.558516,0.827647,0.725494,0.661981
128,0.890316,0.645442,0.060771,0.214993,0.238038,0.788271,0.996142,0.372254,0.824235,0.792425,0.543968,0.831901,0.748899,0.670857
129,0.888026,0.673151,0.065105,0.053026,0.249843,0.792128,0.997106,0.369458,0.830584,0.766479,0.529420,0.836154,0.772303,0.679798
130,0.885736,0.699949,0.069099,0.016760,0.266770,0.795984,0.998071,0.371547,0.836933,0.740532,0.514872,0.840407,0.795708,0.688826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.132632,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.981357,0.451779,0.025853,0.979634,0.911276,0.956241
188,0.099474,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.978694,0.450751,0.019390,0.976725,0.904894,0.965933
189,0.066316,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.976030,0.449723,0.012926,0.973815,0.898512,0.976605
190,0.033158,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.973367,0.448695,0.006463,0.970906,0.892130,0.988214


In [112]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 14)

In [113]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [114]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
#     train, train_val = validation_splitter(train_input, 7)
#     target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
#     optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(36, activation='tanh', 
                                 return_sequences=False), 
                                 input_shape=(train_input.shape[1], 
                                              train_input.shape[2])),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(180, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
#                         validation_data=(train_val,
#                                          target_val),
                        validation_split=0.1,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [115]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [116]:
winner_seed = get_a_good_seed(reshaped_train, 
                              reshaped_target, 
                              reshaped_test, 
                              reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1234552450, 1245412938, 3022067116, 1369529171, 1598313956, 542844551, 2556280518, 3097789028, 3488144078, 312651108]


Step: 0 ___________________________________________
val_loss: 1.2310839891433716
winner_seed: 1234552450


Step: 1 ___________________________________________
val_loss: 1.1677312850952148
winner_seed: 1245412938


Step: 2 ___________________________________________
val_loss: 0.9898253083229065
winner_seed: 3022067116


Step: 3 ___________________________________________
val_loss: 1.1348305940628052


Step: 4 ___________________________________________
val_loss: 0.9152061939239502
winner_seed: 1598313956


Step: 5 ___________________________________________
val_loss: 0.912489652633667
winner_seed: 542844551


Step: 6 ___________________________________________
val_loss: 1.2130768299102783


Step: 7 ___________________________________________
val_loss: 0.9878441095352173


Step: 8 ___________________________________________
val_loss: 0.5890653133392334
winner_seed: 348

In [117]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 111ms/step - loss: 67.9727 - val_loss: 76.3339
Epoch 2/10000
4/4 [==============================] - 0s 7ms/step - loss: 57.9548 - val_loss: 64.3561
Epoch 3/10000
4/4 [==============================] - 0s 7ms/step - loss: 49.0034 - val_loss: 52.7293
Epoch 4/10000
4/4 [==============================] - 0s 7ms/step - loss: 40.2046 - val_loss: 42.0678
Epoch 5/10000
4/4 [==============================] - 0s 7ms/step - loss: 31.9185 - val_loss: 32.5391
Epoch 6/10000
4/4 [==============================] - 0s 7ms/step - loss: 24.8972 - val_loss: 24.2106
Epoch 7/10000
4/4 [==============================] - 0s 7ms/step - loss: 18.9031 - val_loss: 17.2990
Epoch 8/10000
4/4 [==============================] - 0s 7ms/step - loss: 14.0499 - val_loss: 11.8825
Epoch 9/10000
4/4 [==============================] - 0s 7ms/step - loss: 11.1497 - val_loss: 7.7712
Epoch 10/10000
4/4 [==============================] - 0s 7ms/step - loss: 9.0863 - val_los

Epoch 83/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2868 - val_loss: 4.0852
Epoch 84/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2836 - val_loss: 4.0526
Epoch 85/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2978 - val_loss: 4.0768
Epoch 86/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.3261 - val_loss: 4.1571
Epoch 87/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2937 - val_loss: 4.2721
Epoch 88/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.3311 - val_loss: 4.2491
Epoch 89/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.3220 - val_loss: 4.2387
Epoch 90/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2858 - val_loss: 4.2041
Epoch 91/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.3203 - val_loss: 4.2655
Epoch 92/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2638 - val_loss: 4.2320


4/4 [==============================] - 0s 7ms/step - loss: 1.2549 - val_loss: 3.9215
Epoch 165/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2337 - val_loss: 4.0155
Epoch 166/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2947 - val_loss: 3.9891
Epoch 167/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.3417 - val_loss: 3.7603
Epoch 168/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2006 - val_loss: 3.8204
Epoch 169/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2437 - val_loss: 3.9707
Epoch 170/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2474 - val_loss: 4.2628
Epoch 171/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2886 - val_loss: 4.3681
Epoch 172/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.3628 - val_loss: 4.1638
Epoch 173/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2683 - val_loss: 3.8708
Epoch 

4/4 [==============================] - 0s 7ms/step - loss: 1.1754 - val_loss: 3.8880
Epoch 246/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.1290 - val_loss: 3.8579
Epoch 247/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.1513 - val_loss: 3.8018
Epoch 248/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.1350 - val_loss: 3.8323
Epoch 249/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.1686 - val_loss: 3.9326
Epoch 250/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.2771 - val_loss: 4.0701
Epoch 251/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2711 - val_loss: 4.2419
Epoch 252/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2841 - val_loss: 4.4660
Epoch 253/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.2475 - val_loss: 4.3397
Epoch 254/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.3768 - val_loss: 3.7489
Epoch 

4/4 [==============================] - 0s 7ms/step - loss: 1.2297 - val_loss: 4.3809
Epoch 327/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2270 - val_loss: 4.3781
Epoch 328/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2386 - val_loss: 4.2420
Epoch 329/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2344 - val_loss: 4.0939
Epoch 330/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2394 - val_loss: 4.0305
Epoch 331/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2493 - val_loss: 4.0612
Epoch 332/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2336 - val_loss: 4.1524
Epoch 333/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2288 - val_loss: 4.1238
Epoch 334/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2352 - val_loss: 4.1348
Epoch 335/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.2608 - val_loss: 3.9488
Epoch 

...

4/4 [==============================] - 0s 8ms/step - loss: 0.5803 - val_loss: 1.8602
Epoch 1535/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.5730 - val_loss: 1.7852
Epoch 1536/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.5621 - val_loss: 1.0014
Epoch 1537/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.6584 - val_loss: 1.2490
Epoch 1538/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.7372 - val_loss: 1.5379
Epoch 1539/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.5652 - val_loss: 1.7042
Epoch 1540/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.7068 - val_loss: 0.9182
Epoch 1541/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.7781 - val_loss: 1.7793
Epoch 1542/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.7159 - val_loss: 2.0537
Epoch 1543/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.6011 - val_loss: 1.50

4/4 [==============================] - 0s 8ms/step - loss: 0.4931 - val_loss: 1.9972
Epoch 1615/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.4828 - val_loss: 1.9577
Epoch 1616/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.4673 - val_loss: 2.0634
Epoch 1617/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.4395 - val_loss: 2.0580
Epoch 1618/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.4879 - val_loss: 2.1640
Epoch 1619/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.5642 - val_loss: 1.9045
Epoch 1620/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.5548 - val_loss: 1.9339
Epoch 1621/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.5359 - val_loss: 2.0132
Epoch 1622/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.5046 - val_loss: 1.9283
Epoch 1623/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.6255 - val_loss: 2.06

4/4 [==============================] - 0s 8ms/step - loss: 0.4399 - val_loss: 1.8053
Epoch 1695/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.4534 - val_loss: 1.7447
Epoch 1696/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.4355 - val_loss: 1.9576
Epoch 1697/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.4342 - val_loss: 1.8195
Epoch 1698/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.4203 - val_loss: 1.6384
Epoch 1699/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.4421 - val_loss: 1.5959
Epoch 1700/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.4244 - val_loss: 1.7464
Epoch 1701/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.4316 - val_loss: 1.6928
Epoch 1702/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.6256 - val_loss: 1.7505
Epoch 1703/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.4017 - val_loss: 1.86

4/4 [==============================] - 0s 7ms/step - loss: 0.3735 - val_loss: 1.3767
Epoch 1775/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.3973 - val_loss: 1.4169
Epoch 1776/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.3726 - val_loss: 1.4540
Epoch 1777/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.3474 - val_loss: 1.3007
Epoch 1778/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.3446 - val_loss: 1.3185
Epoch 1779/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.3453 - val_loss: 1.2477
Epoch 1780/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.3358 - val_loss: 1.3440
Epoch 1781/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.3280 - val_loss: 1.4181
Epoch 1782/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.3295 - val_loss: 1.4169
Epoch 1783/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.3388 - val_loss: 1.27

4/4 [==============================] - 0s 8ms/step - loss: 0.4378 - val_loss: 0.9667
Epoch 1855/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.4411 - val_loss: 1.0097
Epoch 1856/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.3917 - val_loss: 1.0478
Epoch 1857/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.4554 - val_loss: 1.1977
Epoch 1858/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.5885 - val_loss: 1.0842
Epoch 1859/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.4605 - val_loss: 1.1481
Epoch 1860/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.4367 - val_loss: 1.2011
Epoch 1861/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.4527 - val_loss: 1.2047
Epoch 1862/10000
4/4 [==============================] - 0s 8ms/step - loss: 0.3675 - val_loss: 1.2687
Epoch 1863/10000
4/4 [==============================] - 0s 7ms/step - loss: 0.4031 - val_loss: 1.56

In [118]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [119]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 13ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,9.843187,9.816235,9.840469,9.96563,9.977659,9.891206,9.791612,9.79293,9.724615,9.764837,9.7551,9.657824,9.661549,9.552212,8.881111,8.753441,9.089231,9.442709,8.604382,7.822763,7.469254,7.196555,7.122366,6.979875,6.989618,6.812077,6.801687,6.677431,6.678755,6.605057
Target,7.703,8.956,8.626,6.735,9.161,8.634,7.115,2.781,7.49,7.043,4.877,7.031,5.988,7.275,9.468,7.97,7.866,8.108,9.908,10.723,10.625,10.593,10.681,7.887,9.751,9.654,9.422,11.16,10.491,10.225
Error,2.140187,0.860234,1.214469,3.230629,0.816659,1.257206,2.676612,7.01193,2.234615,2.721837,4.8781,2.626823,3.673549,2.277212,0.586889,0.783441,1.223231,1.334709,1.303618,2.900237,3.155746,3.396446,3.558634,0.907125,2.761382,2.841923,2.620313,4.482569,3.812245,3.619943


In [120]:
display(mae)
display(mape)

2.5636172

0.36782834

In [121]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [122]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[117.821304]] - Target[86.152]| =  Error: [[31.669304]]; MAPE:[[0.367598]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[100.575455]] - Target[107.092]| =  Error: [[6.516548]]; MAPE:[[0.06085]]
1/1 [==============================] - 0s 13ms/step
Ano-5: |Prediction[[40.564625]] - Target[60.703]| =  Error: [[20.138374]]; MAPE:[[0.33175254]]


[array([[31.669304]], dtype=float32),
 array([[6.516548]], dtype=float32),
 array([[20.138374]], dtype=float32)]

19.441408

0.25340018